<a href="https://colab.research.google.com/github/karaage0703/kaggle-book-gokui/blob/main/kaggle_book_gokui_memo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggleに挑む深層学習プログラミングの極意

読書メモです。公式のサポートサイトは以下。

https://github.com/smly/kaggle-book-gokui

## 第1章 機械学習コンテストの基礎知識

省略

## 第2章 探索的データ分析とモデルの作成・検証・性能向上

### 2.1 探索的データ分析

ノーフリーランチ定理：あらゆる問題で性能が高い万能なモデルは存在しない

メモ：基盤モデルによって、ちょっと状況変わってきた気がしなくもない

### 2.2 モデルの作成
#### 2.2.1 ニューラルネットワーク

手書き文字認識データセット読み込み

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data
y = digits.target
print(X.shape, y.shape)

データの中身を確認

https://chusotsu-program.com/matplotlib-digits-plot/

https://karaage.hatenadiary.jp/entry/2018/12/17/073000

In [ ]:
digits.data[0]

In [ ]:
digits.images[0]

In [ ]:
from matplotlib import pyplot as plt 
fig = plt.figure(figsize=(1, 1))
plt.imshow(digits.images[0], cmap=plt.cm.gray_r, interpolation='nearest')
plt.axis('off')
plt.show()

In [ ]:
image_and_labels = list(zip(digits.images, digits.target))

for index, (image, label) in enumerate(image_and_labels[:30]):
    plt.subplot(6, 5, index + 1)
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.axis('off')
    plt.subplots_adjust(wspace=1, hspace=1)
    plt.title('Training: %i' % label)

plt.show()

データ変換

In [ ]:
import torch

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

モデル作成

OptimizerをSGD->Adamに変更すると性能が上がったりします

関連：サポートサイトの[issuesで質問中](https://github.com/smly/kaggle-book-gokui/issues/6)

参考：[【決定版】スーパーわかりやすい最適化アルゴリズム -損失関数からAdamとニュートン法-](https://qiita.com/omiita/items/1735c1d048fe5f611f80)

In [ ]:
from torch import nn, optim

model = nn.Sequential(
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 10),
)
model.train()
lossfun = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
#optimizer = optim.Adam(model.parameters(), lr=0.01)
#optimizer = optim.SGD(model.parameters(), lr=0.02)

In [ ]:
losses = []

for ep in range(100):
    optimizer.zero_grad()
    out = model(X)

    loss = lossfun(out, y)
    loss.backward()

    optimizer.step()

    losses.append(loss.item())

In [ ]:
_, pred = torch.max(out, 1)
print((pred == y).sum().item() / len(y))

In [ ]:
plt.plot(losses)
plt.xlabel('epoch')
plt.ylabel('loss')

ミニバッチ

`batch_size=64`のときは、`X.shape=1797`なので、1エポックの中で`1797/64 = 28`回、学習（パラメータ更新）を繰り返すことになる。

`batch_size=1797`にすると、ミニバッチ処理無しと同じ状況になる。

In [ ]:
import torch
from sklearn.datasets import load_digits
from torch.utils.data import DataLoader, TensorDataset

digits = load_digits()
X = digits.data
y = digits.target

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

dataset = TensorDataset(X, y)
train_loader = DataLoader(dataset, batch_size=64, shuffle=False)

In [ ]:
model = nn.Sequential(
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 10),
)

model.train()
lossfun = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
def train_1epoch(model, train_loader, lossfun, optimizer, device):
    model.train()
    total_loss, total_acc = 0.0, 0.0

    for x, y in tqdm(train_loader):
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        out = model(x)

        loss = lossfun(out, y)
        loss.backward()

        optimizer.step()

        total_loss += loss.item() * x.size(0)

        _, pred = torch.max(out, 1)
        total_acc += torch.sum(pred == y.data)

    avg_loss = total_loss / len(train_loader.dataset)
    avg_acc = total_acc / len(train_loader.dataset)
    return avg_acc, avg_loss

In [ ]:
from tqdm import tqdm
losses = []
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"

for ep in range(100):
    optimizer.zero_grad()
    avg_acc, avg_loss = train_1epoch(
        model, train_loader, lossfun, optimizer, device
    )
    losses.append(avg_loss)

In [ ]:
plt.plot(losses)
plt.xlabel('epoch')
plt.ylabel('loss')

#### 2.2.2 ニューラルネットワーク以外のアルゴリズム

- 勾配ブースティング決定木(GBDT)
- k近傍法(kNN)
- 線形モデル

#### 2.2.3 利用するモデルと得意とするデータセットの種類・性質

#### 2.2.4 ニューラルネットワークの高速化・安定化

[PERFORMANCE TUNING GUIDE](https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html)

#### 2.2.5 ニューラルネットワークの実装における留意点

### 2.3 モデルの検証

#### 2.3.1 汎化性能

https://github.com/smly/kaggle-book-gokui/blob/main/chapter2/02_mlp_hold_out.py

In [ ]:
import torch
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

digits = load_digits()
X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=0
)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_valid = torch.tensor(X_valid, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_valid = torch.tensor(y_valid, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
val_dataset = TensorDataset(X_valid, y_valid)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
import numpy as np
import torch
from tqdm import tqdm

def validate_1epoch(model, val_loader, lossfun, device):
    model.eval()
    total_loss, total_acc = 0.0, 0.0

    with torch.no_grad():
        for x, y in tqdm(val_loader):
            x = x.to(device)
            y = y.to(device)

            out = model(x)
            loss = lossfun(out, y)
            _, pred = torch.max(out, 1)

            total_loss += loss.item() * x.size(0)
            total_acc += torch.sum(pred == y.data)

    avg_loss = total_loss / len(val_loader.dataset)
    avg_acc = total_acc / len(val_loader.dataset)
    return avg_acc, avg_loss


def predict(model, loader, device):
    pred_fun = torch.nn.Softmax(dim=1)
    preds = []
    for x, _ in tqdm(loader):
        with torch.set_grad_enabled(False):
            x = x.to(device)
            y = pred_fun(model(x))
        y = y.cpu().numpy()
        y = np.argmax(y, axis=1)
        preds.append(y)
    preds = np.concatenate(preds)
    return preds

学習・検証

In [ ]:
train_losses = []
valid_losses = []

for ep in range(100):
    train_acc, train_loss = train_1epoch(
        model, train_loader, lossfun, optimizer, device
    )
    valid_acc, valid_loss = validate_1epoch(
        model, val_loader, lossfun, device
    )

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses, label="train_losses")
plt.plot(valid_losses, label="valid_losses")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend()

#### 2.3.2 検証セットが必要な理由

ホールドアウト検証(holdout validation)：訓練セット(train)から検証セット(validation)を切り出し、自分のモデルの性能を検証する

#### 2.3.3 交差検証

データ分割を異なる方法で複数回実施して、それぞれホールドアウト検証をすること。

訓練セットを分割した最小単位を「fold」と呼ぶ

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader, TensorDataset

digits = load_digits()
X = digits.data
y = digits.target

# train と test に分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model = nn.Sequential(
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 10),
)
model = model.to(device)

lossfun = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

cv = KFold(n_splits=5)
oof_train = np.zeros((len(X_train), 10))
train_losses = []
valid_losses = []
train_accs = []
valid_accs = []
test_preds = []

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr, X_val = X_train[train_index], X_train[valid_index]
    y_tr, y_val = y_train[train_index], y_train[valid_index]

    X_tr = torch.tensor(X_tr, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    y_tr = torch.tensor(y_tr, dtype=torch.int64)
    y_val = torch.tensor(y_val, dtype=torch.int64)

    train_dataset = TensorDataset(X_tr, y_tr)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
    val_dataset = TensorDataset(X_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

#### 2.3.4 多様な検証方法の使い分け

stratified k-fold：各foldに含まれる正解ラベルの割合を等しく分割する

group k-fold: データセット内にグループが存在する場合の分割の工夫

stratified group k-fold

multilabel stratified k-fold

time series split

adversarial validation

### 2.4 性能の向上

#### 2.4.1 モデルの複雑性（表現力）

#### 2.4.2 データの拡張

- random erasing
- mixup
- 半教師あり学習

https://qiita.com/karaage0703/items/07157a0406c757ef30b8

https://qiita.com/karaage0703/items/9b56a68a5dba4ad330d9

## 第3章 画像分類入門

### 3.1 畳み込みニューラルネットワークの基礎

### 3.2 コンテスト「Dogs vs. Cats Redux」



### 3.3 最初の学習：CNNアーキテクチャ

### 3.4 最初の学習：データセットの準備と学習ループ



#### 3.4.1 データセットの準備
以下よりコード引用

https://github.com/smly/kaggle-book-gokui/blob/main/chapter3/train.py

引数の`dryrun = True`のときは画像の枚数を１００枚に制限することで、短時間で検証する工夫をしている。

In [ ]:
def setup_train_val_split(labels, dryrun=False, seed=0):
    x = np.arange(len(labels))
    y = np.array(labels)
    splitter = sklearn.model_selection.StratifiedShuffleSplit(
        n_splits=1, train_size=0.8, random_state=seed
    )
    train_indices, val_indices = next(splitter.split(x, y))

    if dryrun:
        train_indices = np.random.choice(train_indices, 100, replace=False)
        val_indices = np.random.choice(val_indices, 100, replace=False)

    return train_indices, val_indices

#### 3.4.2 学習ループ
ループに`tqdm`を使って学習時間の予想を表示できるようにする

### 3.5 最適化アルゴリズムと学習率スケジューリング


#### 3.5.1 最適化アルゴリズム

「とりあえず Momentum SGDとAdamのどちらかを使う」

- Adam: 早く収束してほしい
- Momentum SGD: 汎化性能を高めたい（学習時間がかかる）

※ 上記に反論するような論文もある

#### 3.5.2 学習率・学習期間

学習率は徐々に下げていくのが基本。汎化性能も上がることが多い。

メモ： 学習空間を勾配法でくだっていくイメージをするとなんとなく気持ちを理解できる

学習率のスケジュールとして cosine annealing がある（アニーリングだから、焼きなまし法か）。

### 3.6 データ拡張

#### 3.6.1 random flipとrandom crop

#### 3.6.2 mixup

mixupのような強力なデータ拡張は、学習の初期で利用して途中から利用しないテクニックが有効なことがある。

### 3.7 アンサンブル

#### 3.7.1 交差検証とアンサンブル

- averaging: 各モデルの予測を平均すること
- seedのみ変えたモデルを averagingしても効果がでることが多い
- 交差検証と同時にaveragingをするというテクニックがKaggleでは広く用いられている

#### 3.7.2 test time augmentation(TTA)

推論時のデータ拡張

### 3.8 さらにスコアを伸ばすために


#### 3.8.1 試行錯誤の順番は？

「まず各コンテスト特有の点に優先的に取り組むのがよい」

テクニック的なところは、最後に追い込めるのと、むやみに最初に入力画像サイズを大きくすると、試行錯誤の回数を減らしてしまう

#### 3.8.2 行き詰まった際に、何をする？

エラー分析とか。

具体的には、GradCAMとか？？

#### 3.8.3 論文はどこでどうやって調べる
以下は個人的メモ

[機械学習の論文を探す場所まとめ](https://zenn.dev/karaage0703/articles/46806d77983b8a)